In [1]:
import pandas as pd

In [2]:
# Loading training dataset 
train_df = pd.read_excel('E:\Python\Machine Learning\datasets\golf.xlsx', sheet_name='training')
train_df.shape

(14, 5)

In [3]:
train_df.head()

,OUTLOOK,TEMPERATURE,HUMIDITY,WINDY,class_label
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes


In [4]:
train_df.columns

Index(['OUTLOOK', 'TEMPERATURE', 'HUMIDITY', 'WINDY', 'class_label'], dtype='object')

In [5]:
# Calculate P(E1|O), P(E2|O), P(E3|O) & P(E4|O)
# Calculate P(O)
# Calculate P(E1), P(E2), P(E3) & P(E4)

# To find out : P(O | E1, E2, E3, E4) 
# = [P(E1|O) * P(E2|O) * P(E3|O) * P(E4|O) / (P(E1), P(E2), P(E3) & P(E4))] * P(O)

In [6]:
# Loading test dataset 
test_df = pd.read_excel('E:\Python\Machine Learning\datasets\golf.xlsx', sheet_name='test')
test_df.shape

(1, 5)

In [7]:
test_df.iloc[0]= ["Rainy","Cool","Normal","False"]


ValueError: Must have equal len keys and value when setting with an iterable

In [8]:
test_df

,OUTLOOK,TEMPERATURE,HUMIDITY,WINDY,Unnamed: 4
0,Sunny,Hot,Normal,False,NaN


In [9]:
# number of distinct values of the outcome
len(train_df['class_label'].unique())

2

In [10]:
# the list of distinct values of the outcome 
train_df['class_label'].unique()

array(['No', 'Yes'], dtype=object)

In [41]:
Count_Row=train_df.shape[0] #gives number of row count in the training df
Count_Col=train_df.shape[1] #gives number of col count in the training df

In [42]:
# create an empty df to store propbailities

class_prob_df = pd.DataFrame(columns=['class_label', 'probability'], index=range(len(train_df['class_label'].unique())))
class_prob_df

,class_label,probability
0,NaN,NaN
1,NaN,NaN


In [43]:
train_df['class_label'].value_counts()

Yes    9
No     5
Name: class_label, dtype: int64

In [44]:
features = test_df.columns
features
for feature in features:
    print(feature)

OUTLOOK
TEMPERATURE
HUMIDITY
WINDY


In [45]:
i=0
for val, cnt in train_df['class_label'].value_counts().iteritems():
    print ('value', val, 'was found', cnt, 'times')
    class_prob_df.loc[i].class_label = val
    class_prob_df.loc[i].probability = cnt/Count_Row
    i = i +1
    
class_prob_df

value Yes was found 9 times
value No was found 5 times


,class_label,probability
0,Yes,0.642857
1,No,0.357143


In [46]:
# build the class_prob_df with columns necessary for priori probabilities
proc_class_prob_df = class_prob_df
for feature in features:
    proc_class_prob_df['priori_'+feature+'|outcome'] = ''
    proc_class_prob_df['priori_'+feature] = ''
    
proc_class_prob_df

,class_label,probability,priori_OUTLOOK|outcome,priori_OUTLOOK,priori_TEMPERATURE|outcome,priori_TEMPERATURE,priori_HUMIDITY|outcome,priori_HUMIDITY,priori_WINDY|outcome,priori_WINDY
0,Yes,0.642857,,,,,,,,
1,No,0.357143,,,,,,,,


In [47]:
# calculate the priori probality of events

for idx, row in proc_class_prob_df.iterrows():   
    
    for idx1, row1 in test_df.iterrows():

        for feature in features:
            # get the number of samples for the event
            
            event_samples = train_df[(train_df[feature]==row1[feature])].shape[0]
            event_outcome_samples = train_df[(train_df[feature]==row1[feature]) & (train_df['class_label']==row.class_label)].shape[0]
            outcome_samples = train_df[train_df['class_label']== row.class_label].shape[0]
           
            col_to_update = 'priori_'+feature+'|outcome'
            proc_class_prob_df.at[idx, col_to_update] = event_outcome_samples/outcome_samples
            
            col_to_update = 'priori_'+feature
            proc_class_prob_df.at[idx, col_to_update] = event_samples/Count_Row
            

In [48]:
proc_class_prob_df

,class_label,probability,priori_OUTLOOK|outcome,priori_OUTLOOK,priori_TEMPERATURE|outcome,priori_TEMPERATURE,priori_HUMIDITY|outcome,priori_HUMIDITY,priori_WINDY|outcome,priori_WINDY
0,Yes,0.642857,0.222222,0.357143,0.333333,0.285714,0.666667,0.5,0,0
1,No,0.357143,0.6,0.357143,0.2,0.285714,0.2,0.5,0,0


In [49]:
# calculate posterior probabilties for each outcome
for idx, row in proc_class_prob_df.iterrows():   
    col_to_update = 'posterior_prob'
    posterior_prob = 1
    for feature in features:
        
        col_prior_outcome = 'priori_'+feature+'|outcome'
        
        col_prior = 'priori_'+feature
        
        posterior_prob = posterior_prob * proc_class_prob_df.iloc[idx][col_prior_outcome]/proc_class_prob_df.iloc[idx][col_prior]
        
        posterior_prob = posterior_prob * proc_class_prob_df.iloc[idx]['probability']
        
        
    proc_class_prob_df.at[idx, col_to_update] = posterior_prob
    


ZeroDivisionError: float division by zero

In [20]:
proc_class_prob_df

,class_label,probability,priori_OUTLOOK|outcome,priori_OUTLOOK,priori_TEMPERATURE|outcome,priori_TEMPERATURE,priori_HUMIDITY|outcome,priori_HUMIDITY,priori_WINDY|outcome,priori_WINDY,posterior_prob
0,Yes,0.642857,0.333333,0.357143,0.222222,0.285714,0.666667,0.5,0.666667,0.571429,0.192857
1,No,0.357143,0.4,0.357143,0.4,0.285714,0.2,0.5,0.4,0.571429,0.007143
